In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from pandas_datareader import data as pdr

In [2]:
from pandas_datareader.famafrench import get_available_datasets

In [3]:
get_available_datasets()

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [4]:
ff=pdr.DataReader('F-F_Research_Data_Factors_daily','famafrench',start='1991-1-1')[0]

In [5]:
mm=pdr.DataReader('F-F_Momentum_Factor_daily','famafrench',start='1991-1-1')[0]
mm

,Mom
Date,
1991-01-02,-1.42
1991-01-03,-1.06
1991-01-04,-0.73
1991-01-07,0.26
1991-01-08,0.44
...,...
2023-01-25,0.14
2023-01-26,-1.23
2023-01-27,-2.46


In [6]:
list(mm.columns.values)

['Mom   ']

In [7]:
mm.columns=['Mom']

In [8]:
factors=pd.merge(ff,mm,left_index=True,right_index=True)

In [9]:
factors.head()

,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,
1991-01-02,-0.95,0.64,0.82,0.023,-1.42
1991-01-03,-1.25,0.28,1.17,0.023,-1.06
1991-01-04,-0.24,0.12,0.42,0.023,-0.73
1991-01-07,-1.72,0.32,0.23,0.023,0.26
1991-01-08,-0.29,-0.36,-0.01,0.023,0.44


In [10]:
import yfinance as yf
yf.pdr_override()

In [11]:
tickers=['MSFT','AAPL']

sec_data=pd.DataFrame()

for t in tickers:
    sec_data[t]=pdr.get_data_yahoo(t,start='1991-1-1')['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [12]:
sec_data

,MSFT,AAPL
Date,,
1991-01-02,0.647727,0.312694
1991-01-03,0.652060,0.309101
1991-01-04,0.657476,0.310897
1991-01-07,0.648811,0.310897
1991-01-08,0.636896,0.310897
...,...,...
2023-02-27,250.160004,147.919998
2023-02-28,249.419998,147.410004
2023-03-01,246.270004,145.309998


In [13]:
sec_returns=sec_data/sec_data.shift(1)-1
#Convert all data to percentage
sec_returns=sec_returns.dropna()*100
sec_returns

,MSFT,AAPL
Date,,
1991-01-03,0.668976,-1.149302
1991-01-04,0.830484,0.581323
1991-01-07,-1.317851,0.000000
1991-01-08,-1.836468,0.000000
1991-01-09,0.680353,4.624229
...,...,...
2023-02-27,0.377178,0.824750
2023-02-28,-0.295813,-0.344777
2023-03-01,-1.262928,-1.424602


In [14]:
all=pd.merge(sec_returns,factors,left_index=True,right_index=True)
all.head()

,MSFT,AAPL,Mkt-RF,SMB,HML,RF,Mom
Date,,,,,,,
1991-01-03,0.668976,-1.149302,-1.25,0.28,1.17,0.023,-1.06
1991-01-04,0.830484,0.581323,-0.24,0.12,0.42,0.023,-0.73
1991-01-07,-1.317851,0.000000,-1.72,0.32,0.23,0.023,0.26
1991-01-08,-1.836468,0.000000,-0.29,-0.36,-0.01,0.023,0.44
1991-01-09,0.680353,4.624229,-0.95,0.50,0.06,0.023,0.89


In [15]:
all['MSFT-RF']=all['MSFT']-all['RF']
all.head()

,MSFT,AAPL,Mkt-RF,SMB,HML,RF,Mom,MSFT-RF
Date,,,,,,,,
1991-01-03,0.668976,-1.149302,-1.25,0.28,1.17,0.023,-1.06,0.645976
1991-01-04,0.830484,0.581323,-0.24,0.12,0.42,0.023,-0.73,0.807484
1991-01-07,-1.317851,0.000000,-1.72,0.32,0.23,0.023,0.26,-1.340851
1991-01-08,-1.836468,0.000000,-0.29,-0.36,-0.01,0.023,0.44,-1.859468
1991-01-09,0.680353,4.624229,-0.95,0.50,0.06,0.023,0.89,0.657353


In [16]:
X=all['Mkt-RF']
X.head()

Date
1991-01-03   -1.25
1991-01-04   -0.24
1991-01-07   -1.72
1991-01-08   -0.29
1991-01-09   -0.95
Name: Mkt-RF, dtype: float64

In [17]:
X1=sm.add_constant(X)

reg=sm.OLS(all['MSFT-RF'],X1).fit()

In [18]:
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                MSFT-RF   R-squared:                       0.439
Model:                            OLS   Adj. R-squared:                  0.439
Method:                 Least Squares   F-statistic:                     6321.
Date:                Fri, 03 Mar 2023   Prob (F-statistic):               0.00
Time:                        10:51:02   Log-Likelihood:                -14715.
No. Observations:                8081   AIC:                         2.943e+04
Df Residuals:                    8079   BIC:                         2.945e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0416      0.017      2.502      0.012       0.009       0.074
Mkt-RF         1.1392      0.014     79.506      0.000       1.111       1.167
==============================================================================
Omnibus:                     1281.508   Durbin-Watson:                   1.979
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            23389.935
Skew:                           0.093   Prob(JB):                         0.00
Kurtosis:                      11.333   Cond. No.                         1.17
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [19]:
#Add more factors
X=all[['Mkt-RF','SMB','HML','Mom']]
X.head()

,Mkt-RF,SMB,HML,Mom
Date,,,,
1991-01-03,-1.25,0.28,1.17,-1.06
1991-01-04,-0.24,0.12,0.42,-0.73
1991-01-07,-1.72,0.32,0.23,0.26
1991-01-08,-0.29,-0.36,-0.01,0.44
1991-01-09,-0.95,0.50,0.06,0.89


In [20]:
X1=sm.add_constant(X)

reg=sm.OLS(all['MSFT-RF'],X1).fit()

In [21]:
reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                MSFT-RF   R-squared:                       0.483
Model:                            OLS   Adj. R-squared:                  0.483
Method:                 Least Squares   F-statistic:                     1886.
Date:                Fri, 03 Mar 2023   Prob (F-statistic):               0.00
Time:                        10:51:02   Log-Likelihood:                -14385.
No. Observations:                8081   AIC:                         2.878e+04
Df Residuals:                    8076   BIC:                         2.882e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0498      0.016      3.114      0.002       0.018       0.081
Mkt-RF         1.1289      0.014     79.651      0.000       1.101       1.157
SMB           -0.2931      0.027    -10.999      0.000      -0.345      -0.241
HML           -0.5708      0.024    -23.634      0.000      -0.618      -0.524
Mom           -0.0156      0.018     -0.853      0.394      -0.051       0.020
==============================================================================
Omnibus:                     1351.490   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            28396.153
Skew:                           0.062   Prob(JB):                         0.00
Kurtosis:                      12.183   Cond. No.                         2.08
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""